In [1]:
import pandas as pd
import os
import glob
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
from skfda.representation.grid import FDataGrid
import fdasrsf as fs


import skfda
from skfda.datasets import fetch_growth
from skfda.exploratory.visualization import FPCAPlot
from skfda.preprocessing.dim_reduction import FPCA
from skfda.representation.basis import (
    BSplineBasis,
    FourierBasis,
    MonomialBasis,
)


from __future__ import annotations

from typing import Any, Mapping, Tuple

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import sklearn.cluster
from cartopy.io.img_tiles import GoogleTiles
from matplotlib.axes import Axes
from matplotlib.figure import Figure

from skfda.datasets import fetch_aemet
from skfda.exploratory.depth import ModifiedBandDepth
from skfda.exploratory.visualization import Boxplot, MagnitudeShapePlot
from skfda.exploratory.visualization.fpca import FPCAPlot
from skfda.misc.metrics import l2_distance
from skfda.ml.clustering import KMeans
from skfda.preprocessing.dim_reduction import FPCA

from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from kneed import KneeLocator


from sklearn.datasets import fetch_openml
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import CondensedNearestNeighbour, TomekLinks
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from IPython.display import Image

# Data Cleaning and Time Warping

In [2]:
#read each csv file in current_directory and store it in dataframes

current_directory = os.getcwd()
csv_files = glob.glob(os.path.join(current_directory, 'ABR_exported_files/*.csv'))

dataframes = {}
for file in csv_files:
    filename = os.path.splitext(os.path.basename(file))[0]
    if pd.read_csv(file).shape[1] > 1:
        dataframes[filename] = pd.read_csv(file)
    else:
        dataframes[filename] = pd.read_csv(file, skiprows=2)

In [3]:
subjects = [0,1,2,3,4,5,6,7,8,9,10]
frequencies = [4000, 8000, 16000, 24000, 32000]
dB_levels = [20,25,30,35,40,45,50,55,60,65,70,75,80,85,90]
results_df = pd.DataFrame()

# Iterate over all subject-frequency pairs
for subject in subjects:
    for frequency in frequencies:
        waves = []
        available_dB_levels = []  # List to store the available dB levels for this subject/frequency pairing
        for dB in dB_levels:
            filename = list(dataframes.items())[subject][0]
            df = list(dataframes.values())[subject]
            khz = df[df['Freq(Hz)'] == frequency]
            dbkhz = khz[khz['Level(dB)'] == dB]  # Select the dB level
            if not dbkhz.empty:
                index = dbkhz.index.values[0]
                final = df.iloc[index,48:]
                final = pd.to_numeric(final, errors='coerce')
                wave = final.tolist()[:-1]
                waves.append(wave)
                available_dB_levels.append(dB)  # Add the dB level to the list of available dB levels

        # Align the waves using dynamic time warping
        np_waves = np.array(waves)
        time = np.linspace(0, 1, np_waves.shape[1])
        obj = fs.fdawarp(np_waves.T, time)
        obj.srsf_align(parallel=True)
        aligned_waves = obj.fn.T

        # Iterate over all available dB levels to find the peaks in the aligned waves
        for i, dB in enumerate(available_dB_levels):
            wave = aligned_waves[i]
            wave_after_30 = wave[50:-30]  # start from the 30th point and end before the last 30 points
            peaks, _ = find_peaks(wave_after_30, distance=15)
            peaks = [peak + 50 for peak in peaks]  # adjust the indices
            highest_peaks_indices = sorted(peaks, key=lambda x: wave[x], reverse=True)[:5]
            highest_peaks_indices.sort()
            peak_coordinates = []
            for peak in highest_peaks_indices:
                peak_coordinates.extend([peak, wave[peak]])

            # If fewer than 5 peaks, fill the remaining coordinates with None
            while len(peak_coordinates) < 10:
                peak_coordinates.extend([None, None])

            # Add the results to the dataframe
            new_row = pd.DataFrame([{'Subject': filename, 'Frequency': frequency, 'dB': dB, 'Peak1_X': peak_coordinates[0], 'Peak1_Y': peak_coordinates[1], 'Peak2_X': peak_coordinates[2], 'Peak2_Y': peak_coordinates[3], 'Peak3_X': peak_coordinates[4], 'Peak3_Y': peak_coordinates[5], 'Peak4_X': peak_coordinates[6], 'Peak4_Y': peak_coordinates[7], 'Peak5_X': peak_coordinates[8], 'Peak5_Y': peak_coordinates[9]}])
            results_df = pd.concat([results_df, new_row], ignore_index=True)


Initializing...
Compute Karcher Mean of 8 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 14 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterati

/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homeb

updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 15 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 15 function in SRSF space with lam=0
upd

In [4]:
# remove rows with NA
results_df = results_df[~results_df['Subject'].str.startswith('A')]

#add deaf column
def check_subject(subject):
    if subject.startswith(('B1_1285', 'B2_1291', 'B2_1290')):
        return 1
    else:
        return 0

results_df['Deaf'] = results_df['Subject'].apply(check_subject)

In [5]:
results_df

,Subject,Frequency,dB,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf
0,B1_1282_baseline,4000,55,81,0.00165,112,0.001422,136,0.001335,152,-0.00005,180,0.001415,0
1,B1_1282_baseline,4000,60,66,-0.002112,81,0.002746,112,-0.000114,136,0.003893,191,0.003407,0
2,B1_1282_baseline,4000,65,81,0.00778,112,-0.00104,136,0.003537,151,0.000362,191,0.000339,0
3,B1_1282_baseline,4000,70,81,0.004333,135,0.002188,151,0.002845,191,0.002547,211,0.001529,0
4,B1_1282_baseline,4000,75,81,0.007113,112,0.002525,136,0.006475,151,0.00511,191,0.009667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,B2_1292_baseline,32000,70,79,0.009,100,0.006,117,0.019526,169,0.008,201,0.010877,0
506,B2_1292_baseline,32000,75,79,0.008,117,0.027383,139,0.007,169,0.01,201,0.011,0
507,B2_1292_baseline,32000,80,79,0.013,117,0.030194,139,0.009,169,0.013,200,0.006,0
508,B2_1292_baseline,32000,85,58,0.013,79,0.010292,117,0.031957,169,0.015,201,0.016877,0


In [6]:
# Define the dB thresholds for each subject and frequency
thresholds = {
    'B1': {
        4000: {'1284': 70, '1285': 90, '1286': 80, '1287': 70, '1279': 70, '1280': 80, '1281': 90, '1282': 75},
        8000: {'1284': 45, '1285': 70, '1286': 50, '1287': 30, '1279': 30, '1280': 50, '1281': 65, '1282': 50},
        16000: {'1284': 45, '1285': 60, '1286': 35, '1287': 20, '1279': 25, '1280': 35, '1281': 50, '1282': 35},
        24000: {'1284': 45, '1285': 65, '1286': 40, '1287': 35, '1279': 35, '1280': 40, '1281': 55, '1282': 40},
        32000: {'1284': 50, '1285': 60, '1286': 35, '1287': 30, '1279': 35, '1280': 35, '1281': 70, '1282': 45}
    },
    'B2': {
        4000: {'1283': 70, '1291': 90, '1292': 80, '1290': 80, '1288': 60, '1289': 80},
        8000: {'1283': 30, '1291': 85, '1292': 60, '1290': 75, '1288': 45, '1289': 40},
        16000: {'1283': 20, '1291': 60, '1292': 40, '1290': 60, '1288': 30, '1289': 25},
        24000: {'1283': 35, '1291': 65, '1292': 40, '1290': 70, '1288': 35, '1289': 35},
        32000: {'1283': 35, '1291': 75, '1292': 35, '1290': 90, '1288': 50, '1289': 35}
    }
}

# Add the 'signal' column
def check_signal(row):
    group = row['Subject'].split('_')[0]
    subject = row['Subject'].split('_')[1]
    frequency = row['Frequency']
    dB = row['dB']
    
    # Check if any of the values are nan
    if pd.isnull(group) or pd.isnull(subject) or pd.isnull(frequency) or pd.isnull(dB):
        return np.nan  # or return some other value that makes sense in your context
    
    if dB >= thresholds[group][frequency][subject]:
        return 1
    else:
        return 0


results_df['Signal'] = results_df.apply(check_signal, axis=1)

results_df.head()


,Subject,Frequency,dB,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf,Signal
0,B1_1282_baseline,4000,55,81,0.00165,112,0.001422,136,0.001335,152,-0.00005,180,0.001415,0,0
1,B1_1282_baseline,4000,60,66,-0.002112,81,0.002746,112,-0.000114,136,0.003893,191,0.003407,0,0
2,B1_1282_baseline,4000,65,81,0.00778,112,-0.00104,136,0.003537,151,0.000362,191,0.000339,0,0
3,B1_1282_baseline,4000,70,81,0.004333,135,0.002188,151,0.002845,191,0.002547,211,0.001529,0,0
4,B1_1282_baseline,4000,75,81,0.007113,112,0.002525,136,0.006475,151,0.00511,191,0.009667,0,1


In [7]:
encoded = pd.get_dummies(results_df, columns=['Frequency'])
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
encoded['dB_encoded'] = encoder.fit_transform(encoded[['dB']])
encoded = encoded.drop('dB', axis=1)


In [8]:
encoded

,Subject,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf,Signal,Frequency_4000,Frequency_8000,Frequency_16000,Frequency_24000,Frequency_32000,dB_encoded
0,B1_1282_baseline,81,0.00165,112,0.001422,136,0.001335,152,-0.00005,180,0.001415,0,0,True,False,False,False,False,7.0
1,B1_1282_baseline,66,-0.002112,81,0.002746,112,-0.000114,136,0.003893,191,0.003407,0,0,True,False,False,False,False,8.0
2,B1_1282_baseline,81,0.00778,112,-0.00104,136,0.003537,151,0.000362,191,0.000339,0,0,True,False,False,False,False,9.0
3,B1_1282_baseline,81,0.004333,135,0.002188,151,0.002845,191,0.002547,211,0.001529,0,0,True,False,False,False,False,10.0
4,B1_1282_baseline,81,0.007113,112,0.002525,136,0.006475,151,0.00511,191,0.009667,0,1,True,False,False,False,False,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,B2_1292_baseline,79,0.009,100,0.006,117,0.019526,169,0.008,201,0.010877,0,1,False,False,False,False,True,10.0
506,B2_1292_baseline,79,0.008,117,0.027383,139,0.007,169,0.01,201,0.011,0,1,False,False,False,False,True,11.0
507,B2_1292_baseline,79,0.013,117,0.030194,139,0.009,169,0.013,200,0.006,0,1,False,False,False,False,True,12.0
508,B2_1292_baseline,58,0.013,79,0.010292,117,0.031957,169,0.015,201,0.016877,0,1,False,False,False,False,True,13.0


# Train/Test Split

In [9]:
#train/test split
from sklearn.model_selection import train_test_split
encoded = encoded.dropna()

feature_cols = [col for col in encoded.columns if col not in ['Subject', 'Signal']]
X = encoded[feature_cols]
y = encoded['Signal']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)


# Scaling

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_cols = ['Peak1_X', 'Peak1_Y', 'Peak2_X', 'Peak2_Y', 'Peak3_X', 'Peak3_Y', 'Peak4_X', 'Peak4_Y', 'Peak5_X', 'Peak5_Y']
scaler.fit(x_train[numeric_cols])
x_train[numeric_cols] = scaler.transform(x_train[numeric_cols])
x_test[numeric_cols] = scaler.transform(x_test[numeric_cols])


In [11]:
x_train

,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf,Frequency_4000,Frequency_8000,Frequency_16000,Frequency_24000,Frequency_32000,dB_encoded
13,-0.288209,-0.655185,-0.108490,-0.788855,-0.577037,-0.694173,-1.120402,-0.311083,-0.816412,0.573962,0,False,True,False,False,False,6.0
489,0.947905,-0.291492,0.890831,0.770580,0.969932,0.215773,0.738474,-0.158124,0.329734,-0.223259,0,False,False,False,True,False,9.0
324,1.389375,2.671999,1.033591,2.357710,0.892584,1.251661,0.498619,0.723896,1.017422,1.656907,0,True,False,False,False,False,13.0
193,-0.994561,-0.467775,-1.036430,-0.401905,-1.195825,-0.461058,-0.640692,0.167263,-0.816412,-0.658152,0,False,False,False,False,True,9.0
491,-0.906267,0.589925,-1.393330,0.663991,-1.350522,0.889746,-1.120402,1.188121,-1.962558,-0.225821,0,False,False,False,True,False,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,0.683024,-0.820342,0.177031,-0.295315,0.660538,-0.829521,0.258764,-0.734509,0.176915,-0.406173,1,False,False,False,True,False,6.0
16,-0.288209,0.084273,-0.179870,0.036317,-0.577037,-0.585931,-1.120402,-0.005077,-0.052315,0.322049,0,False,True,False,False,False,9.0
74,-1.082855,-0.543380,-0.536770,-0.452534,-0.731734,0.254127,-1.060439,1.102039,-1.351280,1.908189,0,True,False,False,False,False,12.0
176,-1.082855,-0.291492,-0.465390,-0.749975,-0.267643,-0.583570,-0.340874,-0.271766,-0.281544,-0.380938,0,False,False,False,True,False,7.0


# Deal with Imbalanced Classes with SMOTE

In [29]:
counts = y.value_counts()
print(f"Signal (1): {counts[1]}")
print(f"Noise (0): {counts[0]}")

Signal (1): 285
Noise (0): 156


In [13]:
sm = SMOTE(random_state=13)
X_res, y_res = sm.fit_resample(X, y)
x_train, x_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=13)

# Linear Regression Without Regularization

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

log_reg = LogisticRegression(C=1e10)
log_reg.fit(x_train, y_train)
y_pred = log_reg.predict(x_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.87      0.89      0.88        37
           1       0.92      0.90      0.91        52

    accuracy                           0.90        89
   macro avg       0.89      0.90      0.90        89
weighted avg       0.90      0.90      0.90        89



# Regularized Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

hyperparameters = {'C': [0.001, 0.01, 0.1, 1, 10, 50, 100, 1000], 'penalty': ['l1', 'l2']}
log_reg = LogisticRegression(solver='liblinear')
grid_search = GridSearchCV(estimator=log_reg, param_grid=hyperparameters, cv=5)
grid_search.fit(x_train, y_train)
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_parameters}')
print(f'Best score: {best_score}')


Best parameters: {'C': 100, 'penalty': 'l1'}
Best score: 0.9204828973843059


In [23]:
from sklearn.metrics import classification_report

log_reg_best = LogisticRegression(solver='liblinear', penalty=best_parameters['penalty'], C=best_parameters['C'])
log_reg_best.fit(x_train, y_train)
y_pred = log_reg_best.predict(x_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.87      0.89      0.88        37
           1       0.92      0.90      0.91        52

    accuracy                           0.90        89
   macro avg       0.89      0.90      0.90        89
weighted avg       0.90      0.90      0.90        89



# confusion matrix

In [15]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, columns=['Predicted: No', 'Predicted: Yes'], index=['Actual: No', 'Actual: Yes'])
print(df_cm)

             Predicted: No  Predicted: Yes
Actual: No              33               4
Actual: Yes              5              47


# Coefficients

In [16]:
log_reg_best.coef_

array([[ 1.30364393e+00,  5.51763011e-01,  6.12286275e-03,
         2.33762046e+00, -6.47066443e-01,  3.27821243e+00,
        -8.92489400e-01,  3.50565837e+00, -4.63652882e-01,
         3.01338202e-01, -7.81851624e+00, -6.25066088e+00,
        -1.04503018e+00,  2.61862450e+00,  1.52461848e+00,
         9.62673700e-02,  1.17461769e+00]])

In [17]:
coefficients = pd.DataFrame(log_reg_best.coef_, columns=x_train.columns)
coefficients = coefficients.transpose()
coefficients.reset_index(inplace=True)
coefficients.columns = ['Variable', 'Coefficient']
coefficients


,Variable,Coefficient
0,Peak1_X,1.303644
1,Peak1_Y,0.551763
2,Peak2_X,0.006123
3,Peak2_Y,2.337620
4,Peak3_X,-0.647066
5,Peak3_Y,3.278212
6,Peak4_X,-0.892489
7,Peak4_Y,3.505658
8,Peak5_X,-0.463653
9,Peak5_Y,0.301338
